In [134]:
# Import python libraries
import numpy as np
import pandas as pd
import welleng as we
import plotly.graph_objects as go
from plotly.offline import plot

In [135]:
# Read the Well files
data = pd.read_csv ("well1/well1.csv")

In [136]:
data.head()

,Hole Depth,Bit Depth,On Bottom,On Bottom Hours,On Bottom ROP,Mechanical Specific Energy,Block Height,Hook Load,Trip Speed,Weight on Bit,...,Gravity Toolface,Rig Sub State,Bit Depth.1,MWD Continuous Azi,MWD Continuous Inc,MWD TEMP (deg),Rate Of Penetration,Time Of Penetration,YYYY/MM/DD,HH:MM:SS
0,5565.0,5555.28,0,76.0,0.0,-999.25,4.9,16.3,0.0,0.0,...,134.75,0.0,5555.28,0.0,3735.0,98.44,0.0,0.0,2023/04/03,16:31:30
1,5565.0,5555.28,0,76.0,0.0,-999.25,4.9,16.3,0.0,0.0,...,134.75,0.0,5555.28,0.0,3735.0,98.44,0.0,0.0,2023/04/03,16:31:40
2,5565.0,5555.28,0,76.0,0.0,-999.25,4.9,16.3,0.0,0.0,...,134.75,0.0,5555.28,0.0,3735.0,98.44,0.0,0.0,2023/04/03,16:31:50
3,5565.0,5555.28,0,76.0,0.0,-999.25,4.9,16.2,0.0,0.0,...,134.75,0.0,5555.28,0.0,3735.0,98.44,0.0,0.0,2023/04/03,16:32:00
4,5565.0,5555.28,0,76.0,0.0,-999.25,4.9,16.2,0.0,0.0,...,134.75,0.0,5555.28,0.0,3735.0,98.44,0.0,0.0,2023/04/03,16:32:10


In [137]:
# Assume survey to bit offset is 15m
offset = 15

# Create a new column "Survey Depth" by subtracting 15 from "Bit Depth"
data['Survey Depth'] = data['Bit Depth'] - offset

# Filter out rows before survey starts
survey_data = data[data['Hole Depth'] >= 0]

# Filter out rows before survey starts
survey_data = survey_data[survey_data['Hole Depth'] <= 5701]

# Only save valid survey to dataframe
# Calculate differences in 'inc' and 'azi' from the previous row
survey_data['inc_changed'] = survey_data['Inclination'].diff().ne(0)
survey_data['azi_changed'] = survey_data['Azimuth'].diff().ne(0)

# Filter rows where either 'inc' or 'azi' changed and bit depth increased
filtered_survey_data = survey_data[(survey_data['inc_changed'] & survey_data['azi_changed']) & (survey_data['Bit Depth'].diff() > 0)]

#read md, inc and azi data from dataframe
md_values = filtered_survey_data['Bit Depth'].values
inc_values = filtered_survey_data['Inclination'].values
azi_values = filtered_survey_data['Azimuth'].values

# Create a survey
filtered_survey_data = we.survey.Survey(md=md_values, inc=inc_values, azi=azi_values)


In [138]:
import csv

# Define the filename for the CSV file
csv_filename = 'filtered_survey_data.csv'

# Write the survey data to the CSV file
with open(csv_filename, 'w', newline='') as csvfile:
    fieldnames = ['Bit Depth', 'Inclination', 'Azimuth']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(len(md_values)):
        writer.writerow({'Bit Depth': md_values[i], 'Inclination': inc_values[i], 'Azimuth': azi_values[i]})

print(f"Filtered survey data saved to '{csv_filename}'")


Filtered survey data saved to 'filtered_survey_data.csv'


In [139]:

# Interpolate survey - generate points every 30 meters
survey_data_interp = filtered_survey_data.interpolate_survey(step=30)

# Create a 3D scatter plot
fig = go.Figure()

# Add interpolated survey data as a blue line
fig.add_trace(go.Scatter3d(
    x=survey_data_interp.x,
    y=survey_data_interp.y,
    z=survey_data_interp.z,
    mode='lines',
    line=dict(color='blue'),
    name='survey_interpolated'
))

# Add filtered survey data as red markers
fig.add_trace(go.Scatter3d(
    x=filtered_survey_data.x,
    y=filtered_survey_data.y,
    z=filtered_survey_data.z,
    mode='markers',
    marker=dict(color='red', size=3),  # Adjust marker size here
    name='survey'
))

# Reverse the z-axis for better visualization
fig.update_scenes(zaxis_autorange="reversed")

# Customize the layout
fig.update_layout(
    width=800,  # Set the width of the plot
    height=800,  # Set the height of the plot
    scene=dict(
        xaxis_title='X Axis Title',
        yaxis_title='Y Axis Title',
        zaxis_title='Z Axis Title'
    )
)

# Show the plot
fig.show()